Running using docker vanvalenlab/deepcell-tf:latest-gpu  
Author: Wenwen Zhang, Jiarong Li (Editor), Heqi Wang, Jiao Yuan

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

import tifffile
from PIL import Image

from deepcell.utils.plot_utils import create_rgb_image, make_outline_overlay
from deepcell_toolbox import erode_edges

# from skimage.exposure import rescale_intensity
# from skimage.measure import label, regionprops, regionprops_table

from mesmer_function import *

from deepcell.applications import Mesmer
app = Mesmer()

2024-05-13 14:24:47.634374: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-13 14:24:48.304391: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 20286 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:3b:00.0, compute capability: 8.6


In [2]:
print('Training Resolution:', app.model_mpp, 'microns per pixel')

Training Resolution: 0.5 microns per pixel


### 1. Image segmentation

In [3]:
# The order of the image and the marker_list must be the same
marker_list = ['80ArAr','131Xe','SMA','Collagen_I','Vimentin','CD14','CK19','CD16','CD163','PanCK','CD15',
'PD-L1','CD31','CD45','CD57','CD117','FoxP3','CD4','E-cadherin','CD68','CD11c',
'CD20','CD8A','CD206','Arg1','PD-1','CD66b','GranzymeB','Ki67','CTLA-4','CD3',
'AFP','CD39','CD69','HLA-DR','GPC3','HepPar1','DNA1','DNA2','ICSK1','ICSK2','ICSK3']
# the keys of the dict can be whatever you like
in_nuc_marker_kwargs = {'nuc_channel': ['DNA1']}
in_mem_marker_kwargs = {# 'mem_channel_1': ['HepPar1','AFP','Arg1','GPC3'], 
                        # 'mem_channel_3':['SMA','Vimentin','CD31'],
                        # 'mem_channel_2': ['CD45','CD3', 'CD68',  'CD8A', 'CD66b'],
                        'mem_channel_1': ['ICSK1','ICSK2','ICSK3']
}

# in_mem_marker_kwargs = {'mem_channel_1': ['ICSK1','ICSK2','ICSK3']}

In [4]:
import os
tiff_path='/data/tiff/'
file_ls=os.listdir(tiff_path)

for file in file_ls:
    img_multipledxed=tifffile.imread(tiff_path+file)
    img_norm=Denoise_img(img_multipledxed,98)

    img_mesmer=Generate_mesmer_input(img_norm,marker_list,in_nuc_marker_kwargs,in_mem_marker_kwargs)
    # plot membrane channel and nuclear channel
    # Plot_mesmer_input(img_mesmer)

    # segmentation_predictions=app.predict(img_mesmer,preprocess_kwargs={'percentile':99.9},image_mpp=1,compartment='nuclear')
    try:
        segmentation_predictions=app.predict(img_mesmer,preprocess_kwargs={'percentile':99.9},image_mpp=1,compartment='whole-cell')
    except Exception:
        print(file)
        continue
    
    # plot membrane channel and nuclear channel with segmentation outline
    # rgb_img=create_rgb_image(img_mesmer,channel_colors=['blue', 'green'])
    # Plot_segmentation_result(rgb_img,segmentation_predictions)

    tifffile.imwrite('/data/'+file.split('.')[0]+'_mask.tiff',segmentation_predictions[0,:,:,0])

2024-05-13 14:25:00.689139: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8100


0328_ROI_029_29.tiff
0328_ROI_061_61.tiff


### 2. Segmentation result comparison